In [17]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

# Long format to Wide format

## pandas.DataFrame.pivot_table
- [바로가기](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html?highlight=pivot_table#pandas.DataFrame.pivot_table)
- 함수 원형
```
DataFrame.pivot_table(
    values=None, 
    index=None, 
    columns=None, 
    aggfunc='mean', 
    fill_value=None, 
    margins=False, 
    dropna=True, 
    margins_name='All', 
    observed=False, 
    sort=True
)
```

In [2]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

In [3]:
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [11]:
pd.pivot_table( df, values='D', index='B', columns='C', aggfunc='sum' )

C,large,small
B,,
one,8,6
two,7,12


In [5]:
table = df.pivot_table( values='D', index='B', columns='C', aggfunc='sum' )
table

C,large,small
B,,
one,8,6
two,7,12


## pandas.crosstab
- [바로가기](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html?highlight=crosstab)
- 함수 원형
```
pandas.crosstab(
    index, 
    columns, 
    values=None, 
    rownames=None, 
    colnames=None, 
    aggfunc=None, 
    margins=False, 
    margins_name='All', 
    dropna=True, 
    normalize=False
)
```

In [6]:
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [7]:
pd.crosstab( index=df['B'], columns=df['C'], values=df['D'], aggfunc='sum' )

C,large,small
B,,
one,8,6
two,7,12


# Wide format to Long format

## pandas.DataFrame.melt
- [바로가기](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.melt.html?highlight=melt#pandas.DataFrame.melt)
- 함수원형
```
DataFrame.melt(
    id_vars=None, 
    value_vars=None, 
    var_name=None, 
    value_name='value', 
    col_level=None, 
    ignore_index=True
)
```

In [8]:
table = table.reset_index()
table

C,B,large,small
0,one,8,6
1,two,7,12


In [9]:
table.melt( id_vars='B',value_vars=['large', 'small'] )

,B,C,value
0,one,large,8
1,two,large,7
2,one,small,6
3,two,small,12


# apply Vs. applymap

## apply
- Series, DataFrame, GroupBy
- 함수 원형
```
Series.apply( func, convert_dtype=True, args=(), **kwargs )
DataFrame.apply( func, axis=0, raw=False, result_type=None, args=(), **kwargs )
GroupBy.apply( func, *args, **kwargs )
```

### Series

In [3]:
s = pd.Series(
  [20, 21, 12], 
  index=['London', 'New York', 'Helsinki'] 
)
s

London      20
New York    21
Helsinki    12
dtype: int64

In [6]:
s.apply( lambda x : x ** 2 )

London      400
New York    441
Helsinki    144
dtype: int64

In [7]:
def subtract_custom_value(x, custom_value):
  return x - custom_value

In [9]:
s.apply( subtract_custom_value, args=(5,) )

London      15
New York    16
Helsinki     7
dtype: int64

### DataFrame

In [10]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
df

,A,B
0,4,9
1,4,9
2,4,9


In [16]:
df.apply( np.sum, axis=0 )

A    12
B    27
dtype: int64

In [17]:
df.apply( np.sum, axis=1)

0    13
1    13
2    13
dtype: int64

### GroupBy

In [19]:
df = pd.DataFrame(
{
  'A': 'a a b'.split(),
  'B': [1,2,3],
  'C': [4,6,5]
})
df

,A,B,C
0,a,1,4
1,a,2,6
2,b,3,5


In [22]:
df.groupby('A').apply( lambda x : x.sum() )

,A,B,C
A,,,
a,aa,3,10
b,b,3,5


## applymap
- pandas.DataFrame.applymap
- 함수원형
```
DataFrame.applymap(func, na_action=None, **kwargs)
```

In [23]:
df = pd.DataFrame([[1, 2.12], [3.356, 4.567]])
df

,0,1
0,1.000,2.120
1,3.356,4.567


In [29]:
df.apply( lambda x : x ** 2 )

,0,1
0,1.000000,4.494400
1,11.262736,20.857489


In [28]:
df.applymap( lambda x : x ** 2 )

,0,1
0,1.000000,4.494400
1,11.262736,20.857489


In [36]:
df.apply( lambda x : str(x), axis=0 )

0    0    1.000\n1    3.356\nName: 0, dtype: float64
1    0    2.120\n1    4.567\nName: 1, dtype: float64
dtype: object

In [34]:
df.applymap( lambda x : str(x) )

,0,1
0,1.0,2.12
1,3.356,4.567


# apply Vs. applymap Vs. transform

## transform
- Series, DataFraem, GroupBy
- 함수원형
```
Series.transform(func, axis=0, *args, **kwargs)

```

In [37]:
np.random.seed([3,1415])
df = pd.DataFrame(np.random.randint(10, size=(6, 4)), columns=list('ABCD'))
df

,A,B,C,D
0,0,2,7,3
1,8,7,0,6
2,8,6,0,2
3,0,4,9,7
4,3,2,4,3
5,3,6,7,7


In [38]:
df.apply( lambda x : x / x.std() )

,A,B,C,D
0,0.000000,0.922531,1.825742,1.332785
1,2.213133,3.228859,0.000000,2.665570
2,2.213133,2.767594,0.000000,0.888523
3,0.000000,1.845062,2.347382,3.109832
4,0.829925,0.922531,1.043281,1.332785
5,0.829925,2.767594,1.825742,3.109832


In [ ]:
df.applymap( lambda x : x / x.std() )

In [40]:
df.transform( lambda x : x / x.std() )

,A,B,C,D
0,0.000000,0.922531,1.825742,1.332785
1,2.213133,3.228859,0.000000,2.665570
2,2.213133,2.767594,0.000000,0.888523
3,0.000000,1.845062,2.347382,3.109832
4,0.829925,0.922531,1.043281,1.332785
5,0.829925,2.767594,1.825742,3.109832


In [42]:
df.transform( [np.exp, np.sqrt] )

A                      B                      C            \
           exp      sqrt          exp      sqrt          exp      sqrt   
0     1.000000  0.000000     7.389056  1.414214  1096.633158  2.645751   
1  2980.957987  2.828427  1096.633158  2.645751     1.000000  0.000000   
2  2980.957987  2.828427   403.428793  2.449490     1.000000  0.000000   
3     1.000000  0.000000    54.598150  2.000000  8103.083928  3.000000   
4    20.085537  1.732051     7.389056  1.414214    54.598150  2.000000   
5    20.085537  1.732051   403.428793  2.449490  1096.633158  2.645751   

             D            
           exp      sqrt  
0    20.085537  1.732051  
1   403.428793  2.449490  
2     7.389056  1.414214  
3  1096.633158  2.645751  
4    20.085537  1.732051  
5  1096.633158  2.645751

In [43]:
def subtract_two(x):
  return x['B'] - x['A']

In [46]:
df.apply( subtract_two, axis=1 )

0    2
1   -1
2   -2
3    4
4   -1
5    3
dtype: int64

In [ ]:
# 한 번에 하나의 시리즈만 처리 가능
df.transform( subtract_two, axis=1 )

# get_dummies Vs. OneHotEncoder

## sklearn.preprocessing.OneHotEncoder
- 클래스 원형
```
class sklearn.preprocessing.OneHotEncoder(
  *, categories='auto', drop=None, sparse='deprecated', 
  sparse_output=True, dtype=<class 'numpy.float64'>, 
  handle_unknown='error', min_frequency=None, max_categories=None
)
```

- 주요 파라미터
  - categories: 변환할 범주(list), default='auto'(자동으로 선택)
  - drop: 'first', 'if_binary' 중 선택, default=None
    - 시험 버전에는 'first'만 사용이 가능
    - None: 모든 범주를 그대로 유지
    - first: 첫 번째 범주를 삭제
    - if_binary: 두 개의 범주가 있는 각 열의 첫 번째 범주를 삭제, 1개 또는 2개 이상의 범주는 그대로 유지
    - list: 삭제해야 하는 i 열의 범주
  - handle_unknown: 변환 중에 알 수 없는 범주가 처리되는 방식 지정
    - error: 변환 중 알 수 없는 범주가 있다면 오류 발생(default)
    - ignore: 변환 중 알 수 없는 범주가 있다면 모두 0으로 반환
    - infrequent_if_exist: 변환 중에 알 수 없는 범주가 있다면 가장 드문 범주로 자동으로 매핑
  - min_frequency: 범주가 드문 것으로 간주되는 최소 빈도, default=None


In [3]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
x = [
  ['Male', 1], 
  ['Female', 3], 
  ['Female', 2]
]
x = pd.DataFrame( x, columns=['성별', '순위'] )
x

,성별,순위
0,Male,1
1,Female,3
2,Female,2


In [ ]:
encoder = OneHotEncoder()
encoder.fit(x)
encoder.transform( [['Female', 1], ['Male', 4]] )

In [ ]:
encoder = OneHotEncoder( handle_unknown='ignore' )
encoder.fit(x) 
encoder.transform( [['Female', 1], ['Male', 4]] ).toarray()

In [30]:
encoder = OneHotEncoder()
encoder.fit(x) 
#  encoder.transform( [['Female', 1], ['Male', 2]] ).toarray()
pd.DataFrame( encoder.transform( [['Female', 1], ['Male', 2]] ).toarray(), columns=encoder.get_feature_names() )

,x0_Female,x0_Male,x1_1,x1_2,x1_3
0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0


In [31]:
encoder = OneHotEncoder( drop='first' )
encoder.fit(x) 
pd.DataFrame( encoder.transform( [['Female', 1], ['Male', 2]] ).toarray(), columns=encoder.get_feature_names() )

,x0_Male,x1_2,x1_3
0,0.0,0.0,0.0
1,1.0,1.0,0.0


In [44]:
encoder = OneHotEncoder( drop=['Male', 3] )
encoder.fit(x) 
pd.DataFrame( encoder.transform( [['Female', 1], ['Male', 2]] ).toarray(), columns=encoder.get_feature_names() )

,x0_Female,x1_1,x1_2
0,1.0,1.0,0.0
1,0.0,0.0,1.0
